In [1]:
import asyncio
import json
import os
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Role,
    WorkflowBuilder,
    WorkflowContext,
    ai_function,
    executor,
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")

✅ All imports successful!


## దశ 1: నిర్మిత అవుట్‌పుట్‌ల కోసం Pydantic మోడళ్లను నిర్వచించండి

ఈ మోడళ్లు ఏజెంట్లు తిరిగి పంపే **schema** ను నిర్వచిస్తాయి. Pydantic తో `response_format` ఉపయోగించడం వల్ల:
- ✅ టైప్-సేఫ్ డేటా ఎక్స్ట్రాక్షన్
- ✅ ఆటోమేటిక్ వాలిడేషన్
- ✅ ఫ్రీ-టెక్స్ట్ రెస్పాన్స్‌ల నుండి ఎటువంటి పార్సింగ్ లోపాలు లేవు
- ✅ ఫీల్డ్‌ల ఆధారంగా సులభమైన కండిషనల్ రౌటింగ్


In [2]:
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""

    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""

    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""

    destination: str
    action: str
    message: str


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)


## దశ 2: హోటల్ బుకింగ్ టూల్ సృష్టించండి

ఈ టూల్‌ను **availability_agent** గదులు అందుబాటులో ఉన్నాయా అని తనిఖీ చేయడానికి పిలుస్తుంది. `@ai_function` డెకరేటర్‌ను ఉపయోగించి:
- Python ఫంక్షన్‌ను AI-కాల్ చేయగల టూల్‌గా మార్చండి
- LLM కోసం ఆటోమేటిక్‌గా JSON schema రూపొందించండి
- పారామీటర్ ధృవీకరణను నిర్వహించండి
- ఏజెంట్ల ద్వారా ఆటోమేటిక్ ఆహ్వానాన్ని సాధ్యం చేయండి

ఈ డెమో కోసం:
- **స్టాక్‌హోమ్, సియాటిల్, టోక్యో, లండన్, ఆమ్స్టర్డామ్** → గదులు ఉన్నాయి ✅
- **ఇతర అన్ని నగరాలు** → గదులు లేవు ❌


In [3]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## దశ 3: రూటింగ్ కోసం షరతుల ఫంక్షన్లను నిర్వచించండి

ఈ ఫంక్షన్లు ఏజెంట్ యొక్క ప్రతిస్పందనను పరిశీలించి, వర్క్‌ఫ్లోలో ఏ మార్గాన్ని తీసుకోవాలో నిర్ణయిస్తాయి.

**ప్రధాన నమూనా:**
1. సందేశం `AgentExecutorResponse` గా ఉందో లేదో తనిఖీ చేయండి  
2. నిర్మిత అవుట్‌పుట్ (Pydantic మోడల్) ను పార్స్ చేయండి  
3. రూటింగ్‌ను నియంత్రించడానికి `True` లేదా `False` ను తిరిగి ఇవ్వండి  

వర్క్‌ఫ్లో ఈ షరతులను **ఎడ్జ్‌లపై** మూల్యాంకనం చేసి, తదుపరి ఏ ఎగ్జిక్యూటర్‌ను ఆహ్వానించాలో నిర్ణయిస్తుంది.


In [4]:
def has_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels ARE available.
    
    Returns True if the destination has hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return True  # Default to True if unexpected type

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )

        return result.has_availability
    except Exception as e:
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                <strong>⚠️  Error:</strong> {str(e)}
            </div>
        """)
        )
        return False


def no_availability_condition(message: Any) -> bool:
    """
    Condition for routing when hotels are NOT available.
    
    Returns True if the destination has no hotel rooms.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)

        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )

        return not result.has_availability
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)


## దశ 4: కస్టమ్ డిస్ప్లే ఎగ్జిక్యూటర్ సృష్టించండి

ఎగ్జిక్యూటర్లు వర్క్‌ఫ్లో భాగాలు, ఇవి మార్పులు లేదా సైడ్ ఎఫెక్ట్స్‌ను నిర్వహిస్తాయి. తుది ఫలితాన్ని ప్రదర్శించే కస్టమ్ ఎగ్జిక్యూటర్‌ను సృష్టించడానికి `@executor` డెకరేటర్‌ను ఉపయోగిస్తాము.

**ముఖ్యమైన భావనలు:**
- `@executor(id="...")` - ఒక ఫంక్షన్‌ను వర్క్‌ఫ్లో ఎగ్జిక్యూటర్‌గా నమోదు చేస్తుంది
- `WorkflowContext[Never, str]` - ఇన్‌పుట్/అవుట్‌పుట్ కోసం టైప్ సూచనలు
- `ctx.yield_output(...)` - తుది వర్క్‌ఫ్లో ఫలితాన్ని అందిస్తుంది


In [5]:
@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ display_result executor created with @executor decorator")

✅ display_result executor created with @executor decorator


## దశ 5: పర్యావరణ వేరియబుల్స్ లోడ్ చేయండి

LLM క్లయింట్‌ను కాన్ఫిగర్ చేయండి. ఈ ఉదాహరణ పనిచేస్తుంది:
- **GitHub మోడల్స్** (GitHub టోకెన్‌తో ఉచిత స్థాయి)
- **Azure OpenAI**
- **OpenAI**


In [6]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(base_url=os.environ.get(
    "GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), model_id="gpt-4o")

## దశ 6: నిర్మిత అవుట్‌పుట్‌లతో AI ఏజెంట్లను సృష్టించండి

మేము **మూడు ప్రత్యేక ఏజెంట్లను** సృష్టిస్తాము, ప్రతి ఒక్కటి `AgentExecutor` లో చుట్టబడింది:

1. **availability_agent** - టూల్ ఉపయోగించి హోటల్ అందుబాటును తనిఖీ చేస్తుంది
2. **alternative_agent** - (గదులు లేనప్పుడు) ప్రత్యామ్నాయ నగరాలను సూచిస్తుంది
3. **booking_agent** - (గదులు అందుబాటులో ఉన్నప్పుడు) బుకింగ్ చేయడానికి ప్రోత్సహిస్తుంది

**ప్రధాన లక్షణాలు:**
- `tools=[hotel_booking]` - ఏజెంట్‌కు టూల్‌ను అందిస్తుంది
- `response_format=PydanticModel` - నిర్మిత JSON అవుట్‌పుట్‌ను బలవంతం చేస్తుంది
- `AgentExecutor(..., id="...")` - వర్క్‌ఫ్లో ఉపయోగానికి ఏజెంట్‌ను చుట్టుతుంది


In [7]:
# Agent 1: Check availability with tool
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: Suggest alternative (when no rooms)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 3: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created 3 Agents:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
        </ul>
    </div>
""")
)

## దశ 7: షరతుల ఆధారంగా వర్క్‌ఫ్లో నిర్మించండి

ఇప్పుడు మేము `WorkflowBuilder` ను ఉపయోగించి షరతుల ఆధారంగా రూటింగ్‌తో గ్రాఫ్‌ను నిర్మిస్తాము:

**వర్క్‌ఫ్లో నిర్మాణం:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙         ↘
[no_availability]  [has_availability]
        ↓              ↓
alternative_agent  booking_agent
        ↓              ↓
    display_result ←───┘
```

**ప్రధాన పద్ధతులు:**
- `.set_start_executor(...)` - ప్రవేశ బిందువును సెట్ చేస్తుంది
- `.add_edge(from, to, condition=...)` - షరతుల ఆధారంగా ఎడ్జ్‌ను జత చేస్తుంది
- `.build()` - వర్క్‌ఫ్లోను తుది రూపం ఇస్తుంది


In [8]:
# Build the workflow with conditional routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    # NO AVAILABILITY PATH
    .add_edge(availability_agent, alternative_agent, condition=no_availability_condition)
    .add_edge(alternative_agent, display_result)
    # HAS AVAILABILITY PATH
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Conditional Routing:</strong><br>
            • If <strong>NO availability</strong> → alternative_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result
        </p>
    </div>
""")
)

## దశ 8: టెస్ట్ కేస్ 1 నడపండి - నగరం అందుబాటులో లేకుండా (పారిస్)

మా సిమ్యులేషన్‌లో గదులు లేని పారిస్‌లో హోటళ్లను అడిగి **అందుబాటులో లేకపోవడం** మార్గాన్ని పరీక్షిద్దాం.


In [9]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → alternative_agent → display_result</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], should_respond=True
)

# Run the workflow
events_paris = await workflow.run(request_paris)
outputs_paris = events_paris.get_outputs()

# Display results
if outputs_paris:
    result_paris = AlternativeResult.model_validate_json(outputs_paris[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT (Paris)</h3>
            <div style='background: white; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_paris.alternative_destination}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_paris.reason}</p>
            </div>
        </div>
    """)
    )

## దశ 9: పరీక్షా కేసు 2 నిర్వహించండి - నగరం అందుబాటులో ఉంది (స్టాక్‌హోమ్)

ఇప్పుడు **అందుబాటులో** మార్గాన్ని పరీక్షిద్దాం, మా సిమ్యులేషన్‌లో గదులు ఉన్న స్టాక్‌హోమ్‌లో హోటళ్లను అభ్యర్థించడం ద్వారా.


In [10]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], should_respond=True
)

# Run the workflow
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
            </div>
        </div>
    """)
    )

## ముఖ్యమైన విషయాలు మరియు తదుపరి చర్యలు

### ✅ మీరు నేర్చుకున్నవి:

1. **WorkflowBuilder నమూనా**
   - ప్రవేశ బిందువును నిర్వచించడానికి `.set_start_executor()` ఉపయోగించండి
   - షరతుల రూటింగ్ కోసం `.add_edge(from, to, condition=...)` ఉపయోగించండి
   - వర్క్‌ఫ్లోను తుది రూపం ఇవ్వడానికి `.build()` ను కాల్ చేయండి

2. **షరతుల రూటింగ్**
   - షరతు ఫంక్షన్లు `AgentExecutorResponse` ను పరిశీలిస్తాయి
   - రూటింగ్ నిర్ణయాలను తీసుకోవడానికి నిర్మిత అవుట్‌పుట్‌లను విశ్లేషించండి
   - ఒక ఎడ్జ్‌ను సక్రియం చేయడానికి `True` ను తిరిగి ఇవ్వండి, దాన్ని దాటవేయడానికి `False` ను తిరిగి ఇవ్వండి

3. **టూల్ ఇంటిగ్రేషన్**
   - Python ఫంక్షన్లను AI టూల్స్‌గా మార్చడానికి `@ai_function` ఉపయోగించండి
   - ఏజెంట్లు అవసరమైనప్పుడు టూల్స్‌ను ఆటోమేటిక్‌గా కాల్ చేస్తాయి
   - టూల్స్ JSON ను తిరిగి ఇస్తాయి, దాన్ని ఏజెంట్లు విశ్లేషించగలవు

4. **నిర్మిత అవుట్‌పుట్‌లు**
   - టైప్-సేఫ్ డేటా ఎక్స్ట్రాక్షన్ కోసం Pydantic మోడల్స్ ఉపయోగించండి
   - ఏజెంట్లను సృష్టించినప్పుడు `response_format=MyModel` ను సెట్ చేయండి
   - `Model.model_validate_json()` తో రెస్పాన్స్‌లను విశ్లేషించండి

5. **కస్టమ్ ఎగ్జిక్యూటర్లు**
   - వర్క్‌ఫ్లో భాగాలను సృష్టించడానికి `@executor(id="...")` ఉపయోగించండి
   - ఎగ్జిక్యూటర్లు డేటాను మార్చవచ్చు లేదా సైడ్ ఎఫెక్ట్‌లను నిర్వహించవచ్చు
   - వర్క్‌ఫ్లో ఫలితాలను ఉత్పత్తి చేయడానికి `ctx.yield_output()` ఉపయోగించండి

### 🚀 వాస్తవ ప్రపంచ అనువర్తనాలు:

- **ట్రావెల్ బుకింగ్**: లభ్యతను తనిఖీ చేయండి, ప్రత్యామ్నాయాలను సూచించండి, ఎంపికలను పోల్చండి
- **కస్టమర్ సర్వీస్**: సమస్య రకం, భావోద్వేగం, ప్రాధాన్యత ఆధారంగా రూట్ చేయండి
- **ఈ-కామర్స్**: స్టాక్‌ను తనిఖీ చేయండి, ప్రత్యామ్నాయాలను సూచించండి, ఆర్డర్‌లను ప్రాసెస్ చేయండి
- **కంటెంట్ మోడరేషన్**: టాక్సిసిటీ స్కోర్లు, యూజర్ ఫ్లాగ్‌ల ఆధారంగా రూట్ చేయండి
- **అప్రూవల్ వర్క్‌ఫ్లోలు**: మొత్తం, యూజర్ రోల్, రిస్క్ లెవెల్ ఆధారంగా రూట్ చేయండి
- **మల్టీ-స్టేజ్ ప్రాసెసింగ్**: డేటా నాణ్యత, పూర్తి స్థాయిపై ఆధారపడి రూట్ చేయండి

### 📚 తదుపరి చర్యలు:

- మరింత సంక్లిష్టమైన షరతులను జోడించండి (బహుళ ప్రమాణాలు)
- వర్క్‌ఫ్లో స్టేట్ మేనేజ్‌మెంట్‌తో లూప్‌లను అమలు చేయండి
- పునర్వినియోగపరచదగిన భాగాల కోసం సబ్-వర్క్‌ఫ్లోలను జోడించండి
- నిజమైన APIలతో (హోటల్ బుకింగ్, ఇన్వెంటరీ సిస్టమ్‌లు) ఇంటిగ్రేట్ చేయండి
- లోపాల నిర్వహణ మరియు ఫాల్బ్యాక్ మార్గాలను జోడించండి
- బిల్ట్-ఇన్ విజువలైజేషన్ టూల్స్‌తో వర్క్‌ఫ్లోలను విజువలైజ్ చేయండి


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**విమర్శ**:  
ఈ పత్రాన్ని AI అనువాద సేవ [Co-op Translator](https://github.com/Azure/co-op-translator) ఉపయోగించి అనువదించారు. మేము ఖచ్చితత్వానికి ప్రయత్నిస్తున్నప్పటికీ, ఆటోమేటెడ్ అనువాదాలలో తప్పులు లేదా అసమానతలు ఉండవచ్చు. దయచేసి, దాని స్వదేశీ భాషలోని అసలు పత్రాన్ని అధికారం కలిగిన మూలంగా పరిగణించండి. కీలకమైన సమాచారం కోసం, ప్రొఫెషనల్ మానవ అనువాదాన్ని సిఫారసు చేస్తాము. ఈ అనువాదాన్ని ఉపయోగించడం వల్ల కలిగే ఏవైనా అపార్థాలు లేదా తప్పుదారులు కోసం మేము బాధ్యత వహించము.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
